In [1]:
# Read google sheet file into pandas dataframe using csv format
# Example: https://stackoverflow.com/questions/19611729/getting-google-spreadsheet-csv-into-a-pandas-dataframe

import pandas as pd
import openpyxl
from openpyxl.styles import Font, Alignment, Border, Side
from openpyxl.utils import get_column_letter, column_index_from_string
import datetime



In [2]:
# Read data from google sheet
curricula = pd.read_csv('https://docs.google.com/spreadsheets/d/' + 
                   '1x-wUM_eXMLmJGC289DvRmF7nFZvdAo4x9cgIArV0fg0' +
                   '/export?gid=579824879&format=csv')

programmes = pd.read_csv('https://docs.google.com/spreadsheets/d/' + 
                   '1x-wUM_eXMLmJGC289DvRmF7nFZvdAo4x9cgIArV0fg0' +
                   '/export?gid=736085643&format=csv')
# set index to the first column
programmes.set_index('Unnamed: 0', inplace=True)




In [3]:
# From programme data, list colum names where data is not missing and not unnamed
prog_ids = programmes.columns[programmes.notna().any() & ~programmes.columns.str.contains('unnamed', case=False)].tolist()

# Select the secont column name as target for the example
target = prog_ids[1]


'MI BSc'

In [ ]:
# Filter curricula where cells in the target column contain "KÖT"
curricula_kot = curricula[curricula[target].str.contains('KÖT', na=False)]
# Keep only the folowing columns in the folowing order: 'Munkanév', 'Rövidítés', 'Kredit E', 'Kredit Gy', 'E (ó/ hét)', 'Gy (ó/ hét)', 'Előfeltétel'
curricula_kot = curricula_kot[['Munkanév', 'Rövidítés', 'Kredit E', 'Kredit Gy', 'E (ó/ hét)', 'Gy (ó/ hét)', 'Előfeltétel']]


In [6]:
# Define a function to merge cells in a row
def merge_row(rowindex: int, columns: int, sheet):
        sheet.merge_cells(start_row=rowindex, start_column=1, end_row=rowindex, end_column=columns)

# Define a function to get semester count
def get_semester_count(target, programmes) -> int:
    return int(programmes.loc['Félév', target])


# Define a function to output the header
def header(rowindex: int, columns: int, sheet, target, programmes) -> int:
    # Collect basic info for the target programme
    programme_name = programmes.loc['Szak', target]
    programme_leader = programmes.loc['Szakfelelős', target]
    sheet['A' + str(rowindex)] = f'{programme_name}, szakfelelős: {programme_leader}'
    merge_row(rowindex, columns, sheet)
    sheet['A' + str(rowindex)].font = Font(bold=True)
    
    programme_semesters = programmes.loc['Félév', target]
    programme_credits = programmes.loc['Összes kredit', target]
    sheet['A' + str(rowindex + 1)] = f'{programme_semesters} félév, {programme_credits} kredit'
    merge_row(rowindex + 1, columns, sheet)
    
    return rowindex+2

In [7]:
# Define a function to output a semester
def semester(rowindex: int, columns: int, sheet, semester: int, target, curricula) -> int:
    # Output semester title
    rowindex = rowindex + 1 # Add an empty row
    sheet['A' + str(rowindex)] = f'{semester}. félév'
    merge_row(rowindex, columns, sheet)
    sheet['A' + str(rowindex)].font = Font(bold=True)
    sheet['A' + str(rowindex)].alignment = Alignment(horizontal='center')
    rowindex = rowindex + 1 # Add an empty row
    
    # Output the column headers
    min_row = rowindex
    headers = ['Tárgy neve', 'Tárgy kód', 'Előadás Kr.', 'Gyakorlat Kr.', 'Előadás óra', 'Gyakorlat óra', 'Előteltétel']
    for i, header in enumerate(headers):
        sheet[get_column_letter(i+1) + str(rowindex)] = header
        sheet[get_column_letter(i+1) + str(rowindex)].font = Font(bold=True)
        sheet[get_column_letter(i+1) + str(rowindex)].alignment = Alignment(wrap_text=True, horizontal='center', vertical='top')
    rowindex = rowindex + 1 # Next row
    
    # Get the curricula for the semester, where the target column starts with the semester number +"KÖT"
    curricula_kot = curricula[curricula[target].str.contains(f'{semester}-KÖT', na=False)]
    # Keep only the folowing columns in the folowing order: 'Munkanév', 'Rövidítés', 'Kredit E', 'Kredit Gy', 'E (ó/ hét)', 'Gy (ó/ hét)', 'Előfeltétel'
    curricula_kot = curricula_kot[['Munkanév', 'Rövidítés', 'Kredit E', 'Kredit Gy', 'E (ó/ hét)', 'Gy (ó/ hét)', 'Előfeltétel']]
    
    # Output the curricula
    for _, row in curricula_kot.iterrows():
        for j, value in enumerate(row):
            sheet[get_column_letter(j+1) + str(rowindex)] = value
            if (j == 0 or j == 6):
                sheet[get_column_letter(j+1) + str(rowindex)].alignment = Alignment(wrap_text=True, vertical='top')
            else:
                sheet[get_column_letter(j+1) + str(rowindex)].alignment = Alignment(horizontal='center', vertical='top')
        rowindex += 1
    
    

    # Sum the credits and hours
    for i, col in enumerate(['C', 'D', 'E', 'F']):
        sheet[col + str(rowindex)] = f'=SUM({col}{min_row+1}:{col}{rowindex-1})'
        sheet[col + str(rowindex)].font = Font(bold=True)
        sheet[col + str(rowindex)].alignment = Alignment(horizontal='center')
    
    # Summarize the semester with number of subjects and sum of credits
    sheet['A' + str(rowindex)] = f'=CONCATENATE("Összesen {curricula_kot.shape[0]} tárgy, ", SUM(C{min_row+1}:D{rowindex-1}), " kr.")'
    sheet['A' + str(rowindex)].font = Font(bold=True)
  
    
    # Set table borders
    for row in sheet.iter_rows(min_row=min_row, max_row=rowindex, min_col=1, max_col=columns):
        for cell in row:
            cell.border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))  
    
    
    return rowindex




In [8]:
# Function to output the curricula of a programme
def output_curricula(target, programmes, curricula):
    # Generate filename containing the target programme name and the current date
    filename = f'{target}_{datetime.datetime.now().strftime("%Y%m%d")}.xlsx'
    
    df = pd.DataFrame()
    
    writer = pd.ExcelWriter(filename, engine='openpyxl')
    df.to_excel(writer, sheet_name='Tanterv', index=False)
    workbook = writer.book
    ws = writer.sheets['Tanterv']
    
    
    workbook.calculation.on_demand = True
    workbook.enable_calculation = True
    
    columns = 7
    columnwidths = [40, 15, 10, 10, 10, 10, 20]
    # Set table column width
    for i, width in enumerate(columnwidths):
        ws.column_dimensions[get_column_letter(i+1)].width = width
    
    rowindex = header(1, columns, ws, target, programmes)
    
    for i in range(1, get_semester_count(target, programmes)+1):
        rowindex = semester(rowindex, columns, ws, i, target, curricula)+1
    
    # Save the workbook
    writer.save()

In [9]:
# For each programme, output the curricula
for target in prog_ids:
    output_curricula(target, programmes, curricula)
